In [46]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import functions_forcrawl
edit = functions_forcrawl.All_func()

def clean_semicolon(st):
    
    if re.search(';$|; $', st, flags=re.S):
        return re.sub(';$|; $', '', st, flags=re.S).strip()
    
    else:
        return st
    
def add_minutes(t, m):
    
    hours, minutes = map(int, t.split(":"))
    total_minutes = 60 * hours + minutes + m
    new_hours = total_minutes // 60
    new_minutes = total_minutes % 60
    return "{:02d}:{:02d}".format(new_hours % 24, new_minutes)


import requests

cookies = {
    'PHPSESSID': 'lbwww6~2f22d77a3de3da12104acedee740dc52',
    'joe-chnlcustid': '0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
}

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'PHPSESSID=lbwww6~2f22d77a3de3da12104acedee740dc52; joe-chnlcustid=0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
    'Origin': 'https://online.eccmid.org',
    'Referer': 'https://online.eccmid.org/programme-live-1?coday=2023-02-16&embed=&typeHideAllBut=15,11',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'idCat': '1',
    'sessionRef': 'S74',
    'timezone': 'Europe/Paris',
    'defaultTimezone': 'Europe/Paris',
    'dispCountry': '',
    'target': '_self',
    'embed': 'false',
    'dispCities': '',
    'firstnameFull': '',
    'page': '',
}

response = requests.post(
    'https://online.eccmid.org/fo-user-display-session-details.php',
    cookies=cookies,
    headers=headers,
    data=data,
)

dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_number':[],
    'name':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

dates = ['02-16', '03-15', '03-16', '04-15', '04-16', '04-17', '04-18']


In [47]:
link = 'https://online.eccmid.org/programme-live-1?coday=2023-03-15&embed=&typeHideAllBut=15,11'

In [48]:
se_no = 1
for dat in dates:
    print(dat)
    reqout = requests.get(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&typeHideAllBut=15,11',
                         headers=headers,
                         cookies=cookies)
    soup = BS(reqout.content, 'lxml')
    
    outer_row = soup.find_all(class_='row live-program-card-row-grid')
    print(len(outer_row))
    locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')
    
    for ou in outer_row:

        col_div = 0
        print(len(ou.find_all(class_='border text-center program-grid-calandar')))
        
        for i_id in ou.find_all(class_='border text-center program-grid-calandar'):
            
            if i_id.find(class_='programme-session-card programme-session-display border'):
                
                session_number = f'S{se_no}'
                print(f'=====**====={session_number}=====**=======', col_div)
                i_id = i_id.find(class_='programme-session-card programme-session-display border')

                data['sessionRef'] = i_id.get('data-session-reference')
                
                req = requests.post(
                    'https://online.eccmid.org/fo-user-display-session-details.php',
                    cookies=cookies,
                    headers=headers,
                    data=data)

                soupin = BS(req.content, 'lxml')
                sess_data = soupin.find(id='modal-session-content')

                # -------------------------------------    date time location ya type     ------------------------------------
                sess_date = ''
                sess_time = ''
                sess_loc = ''
                sess_type = ''
                sess_loc = ''
                sess_loc = locations[col_div].text.strip()
                date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                sess_date = date_time_type.split(',')[0].strip()
                sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                sess_type = date_time_type.split('CET')[-1].strip()

                # --------------------------------------  category or type and title -----------------------------------------------------
                sess_cate = ''
                sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                # --------------------------------------- abstract text OR co organised ---------------------------------------------
                co_org = ''
                sess_text = ''
                if sess_data.find(class_='modal-session-organized-by mt-2'):
                    co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                if sess_data.find(class_='modal-session-desc mt-2'):
                    sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()
                    
                sess_text = co_org + ' ' + sess_text

                # ------------------------------------------- session ka authors aff --------------------------------------------------
                sess_author = ''
                sess_aff = ''

                if sess_data.find(class_='modal-session-moderators'):
                    for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                        aff = i.find(class_='modal-session-moderator-country').text.strip()
                        auth = i.text.strip()
                        auth = auth.replace(aff, '').split('-')[0]
                        aff = aff.replace(',', '').strip()

                        sess_author += auth + '; '
                        sess_aff += aff + '; '

                sess_author = clean_semicolon(sess_author)
                sess_aff = clean_semicolon(sess_aff)
                sess_date = sess_date + '/2023'

                print(f'sess date :- {sess_date}')
                print(f'sess time :- {sess_time}')
                print(f'sess loca :- {sess_loc}')
                print(f'sess type :- {sess_type}')
                print(f'sess cate :- {sess_cate}')
                print(f'sess title :- {sess_title}')
                print(f'abs text :- {sess_text}')
                print(f'sess auth :- {sess_author}')
                print(f'sess aff :- {sess_aff}')

                dic['session_number'].append(session_number)
                dic['name'].append('Session')
                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(sess_title)
                dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&typeHideAllBut=15,11')
                dic['authors'].append(sess_author)
                dic['author_affiliation'].append(sess_aff)
                dic['abstract_text'].append(sess_text)
                dic['date'].append(sess_date)
                dic['start_time'].append(sess_time.split('-')[0].strip())
                dic['end_time'].append(sess_time.split('-')[-1].strip())
                dic['location'].append(sess_loc)
                dic['session_title'].append(sess_title)
                dic['session_type'].append(sess_type)
                dic['category'].append(sess_cate)
                dic['sub_category'].append('')
                dic['disclosure'].append('')
                print('======'*20)
                
                # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*
                
                abs_data = sess_data.find(class_='modal-sessions-interventions')
                if abs_data:
                    
                    pno = 1
                    for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                        abs_title = ab.find(style='font-weight: bold').text.strip()

                        # abs author
                        abs_author = ''
                        abs_aff = ''
                        if ab.find(class_='modal-session-faculties mb-1'):
                            ab_authaff = ab.find(class_='modal-session-faculties mb-1').text.strip()
                            abs_author = ', '.join(ab_authaff.split(',')[:-1])
                            abs_aff = ab_authaff.split(',')[-1]

                        if abs_title == '':
                            abs_title = 'TBD'
                            
                        # source id...... time shit
                        
                        ar_id = ''
                        art_time = ''
                        
                        if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                            src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                            if src_code:

                                if re.search('^\d{1,6}', src_code):
                                    ar_id = src_code + '-' + str(pno)
                                    art_time = ''
                                elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                    str_pno = str(pno)
                                    if len(str_pno) == 1:
                                        str_pno = '0' + str_pno
                                    ar_id = src_code + str_pno
                                    art_time = sess_time
                        else:
                            ar_id = 'QC'
                        
                            
                        print(ar_id, '===', art_time)
                        print(f'abs title :- {abs_title}')
                        print(f'abs auth :- {abs_author}')
                        print(f'abs aff :- {abs_aff}')

                        dic['session_number'].append(session_number)
                        dic['name'].append('Abstract')
                        dic['source_id'].append(ar_id)
                        dic['manual_id'].append('')
                        dic['article_title'].append(abs_title)
                        dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&typeHideAllBut=15,11')
                        dic['authors'].append(abs_author)
                        dic['author_affiliation'].append(abs_aff)
                        dic['abstract_text'].append('')
                        dic['date'].append(sess_date)
                        dic['start_time'].append('')
                        dic['end_time'].append('')
                        dic['location'].append(sess_loc)
                        dic['session_title'].append(sess_title)
                        dic['session_type'].append(sess_type)
                        dic['category'].append(sess_cate)
                        dic['sub_category'].append('')
                        dic['disclosure'].append('')
                        print('-*-+-'* 17)
                        pno += 1
                        
                se_no += 1
                print('inr', se_no)
                
            col_div += 1
            

02-16
3
1
=====**=====S1=====**======= 0
sess date :- 16/02/2023
sess time :- 15:30 - 16:00
sess loca :- Industry Track
sess type :- Industry SessionIndustry Track
sess cate :- 11. Other
sess title :- S74 - NMR-based lipoprotein phenotyping reveals severity and recovery subgroups in populations infected by SARS-CoV-2
abs text :- Co-organised with                    
Bruker Microbiology & Diagnostics 
sess auth :- 
sess aff :- 
S7401 === 15:30 - 16:00
abs title :- TBD
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
inr 2
1
1
03-15
3
1
=====**=====S2=====**======= 0
sess date :- 15/03/2023
sess time :- 15:30 - 16:00
sess loca :- Industry Track
sess type :- Industry SessionIndustry Track
sess cate :- 11. Other
sess title :- S75 - Management of ESBL infections in the post-MERINO era
abs text :- Co-organised with                    
MSD 
sess auth :- 
sess aff :- 
inr 3
1
1
03-16
2
1
=====**=====S3=====**======= 0
sess date :- 1

sess date :- 15/04/2023
sess time :- 13:30 - 15:30
sess loca :- Hall E
sess type :- Integrated SymposiumHall E
sess cate :- 11. Other
sess title :- S6 - Emerging evidence on the use of cefiderocol in Gram-negative infections caused by non-fermentative bacteria
abs text :- Organized by                    
Shionogi Over the last few years a range of new antibiotics has become available for the treatment of difficult-to-treat-resistant Gram-negative infections. In this Integrated Symposium we will explore the challenges facing clinicians when managing these infections and discuss the latest emerging evidence on the use of cefiderocol on the treatment of infections caused by non-fermentative Gram-negative bacteria, such as P. aeruginosa and A. baumannii.
sess auth :- 
sess aff :- 
inr 11
=====**=====S11=====**======= 2
sess date :- 15/04/2023
sess time :- 13:30 - 15:00
sess loca :- Hall F
sess type :- Integrated SymposiumHall F
sess cate :- 11. Other
sess title :- S7 - The rise of invasive

sess date :- 15/04/2023
sess time :- 16:15 - 18:15
sess loca :- Hall H
sess type :- Integrated SymposiumHall H
sess cate :- 11. Other
sess title :- S9 - TBC
abs text :- Organized by                    
Pfizer TBC
sess auth :- 
sess aff :- 
inr 18
5
5
5
04-16
12
5
=====**=====S18=====**======= 0
sess date :- 16/04/2023
sess time :- 08:30 - 10:30
sess loca :- Hall D
sess type :- Integrated SymposiumHall D
sess cate :- 11. Other
sess title :- S16 - Diagnosis and management of refractory and rare moulds: a case-based approach
abs text :- Organized by                    
F2G (in partnership with Shionogi) How do you define refractory IFD? Different centres use different approaches, but an expert consensus has produced a guidance document to support clinicians and clinical triallists. One reason why a case may be refractory is that the causative organism is a rare mould and not the typical invasive aspergillosis which most clinicians would expect. This session will provide guidance on how to

sess date :- 16/04/2023
sess time :- 12:15 - 13:15
sess loca :- Hall H
sess type :- Integrated SymposiumHall H
sess cate :- 11. Other
sess title :- S20 - Building a strong defense: addressing the importance of adult vaccination in the evolving pneumococcal disease landscape
abs text :- Organized by                    
Pfizer While the introduction of pneumococcal conjugate vaccines has resulted in a reduction in pneumococcal disease through both direct and indirect effects, a substantial burden of disease remains in adults. Now more than ever, there is a need to help reduce vaccine preventable respiratory diseases and their impact on healthcare resource use. In this symposium we will discuss pneumococcal conjugate vaccines and how expanded valent vaccines can help protect adults against serotypes that are important causes of pneumococcal disease.
sess auth :- Joanne M LANGLEY
sess aff :- Canada
inr 26
5
=====**=====S26=====**======= 0
sess date :- 16/04/2023
sess time :- 13:30 - 15:30


sess date :- 16/04/2023
sess time :- 16:15 - 18:15
sess loca :- Hall F
sess type :- Integrated SymposiumHall F
sess cate :- 11. Other
sess title :- S32 - AMR, sustainability and efficiency: 'Staying ahead of tomorrow's challenges, today'
abs text :- Organized by                    
Thermo Fisher Scientific Microbiology laboratories are faced with the consequences of a number of global health crises, including the ongoing spread of antimicrobial resistance (AMR), and newer emerging resistance in the aftermath of the coronavirus pandemic. On top of this, laboratories are often challenged by increasingly higher demands and limited resources, with pressure to streamline workflows and operate more sustainably. So how can microbiology labs stay ahead of tomorrow's challenges, today?

In this session, we will explore:
.	How antimicrobial susceptibility testing (AST) devices featuring new antimicrobials can be a promising tool in the battle against AMR
.	How artificial intelligence-enabled tec

sess date :- 17/04/2023
sess time :- 12:15 - 13:15
sess loca :- Hall E
sess type :- Integrated SymposiumHall E
sess cate :- 11. Other
sess title :- S39 - BacteriologIA: Artificial Intelligence applied to bacteriology
abs text :- Organized by                    
Copan Group Technological innovation is growing in the healthcare system. 
The microbiology laboratory has, like never before, the opportunity to change. The introduction of lab automation, digitalisation, and Artificial Intelligence can aid the microbiologist in managing the multiple challenges that a modern microbiology laboratory faces: staff reduction, inquiry of traceability, improved quality, reduction of turn-around time, and the importance of demonstrating the clinical value brought by the microbiology lab to the patient.  
In this symposium, we will present the experiences of clinical laboratories that re-shaped their workflows according to the adoption of new technologies.
sess auth :- Derek MACFADDEN
sess aff :- Canad

sess date :- 17/04/2023
sess time :- 16:15 - 18:15
sess loca :- Hall D
sess type :- Integrated SymposiumHall D
sess cate :- 11. Other
sess title :- S58 - Spanning the diagnostics portfolio: an in depth look at outcomes and use cases benefitting different patient populations
abs text :- Organized by                    
Roche In this symposium, there will be an engaging divergence of diagnostics and how they are utilised in different patient populations. During the first session, the speaker will focus on all comers with bloodstream infections and the results of a randomised control trial in France. Medical and economic outcomes were generated comparing rapid molecular blood culture identification testing to traditional microbiological techniques in order to assess how a rapid syndromic result can help to improve mortality and decrease both time to appropriate therapy and length of stay, while reducing costs.

In the second half, the speakers will discuss humoral and cellular immune resp

sess date :- 18/04/2023
sess time :- 12:15 - 13:15
sess loca :- Hall D
sess type :- Integrated SymposiumHall D
sess cate :- 11. Other
sess title :- S14 - Rapid AST on positive blood culture: efficient use in clinical routine laboratories
abs text :- Organized by                    
QUANTAMATRIX The aim of our Integrated Symposium will be to focus on different topics related to medico-economical aspects of rapid AST solutions and how it can bring added value for different laboratory workflow: How rapid AST solutions can be economically valuable for a clinical laboratory. Medico-economical contrast between public and private laboratory environment for rapid AST solutions. Use of rapid AST solutions in different laboratory workflow schedules
sess auth :- Gérald ULRICH ; Paul Christoffer LINDEMANN
sess aff :- France; Norway
S1401 === 12:15 - 13:15
abs title :- Welcome and introduction
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--

In [49]:
df = pd.DataFrame(dic)
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_title,session_number,name,session_type,category,sub_category,disclosure
0,,,S74 - NMR-based lipoprotein phenotyping reveal...,https://online.eccmid.org/programme-live-1?cod...,,,Co-organised with \nBruker ...,16/02/2023,15:30,16:00,Industry Track,S74 - NMR-based lipoprotein phenotyping reveal...,S1,Session,Industry SessionIndustry Track,11. Other,,
1,S7401,,TBD,https://online.eccmid.org/programme-live-1?cod...,,,,16/02/2023,,,Industry Track,S74 - NMR-based lipoprotein phenotyping reveal...,S1,Abstract,Industry SessionIndustry Track,11. Other,,
2,,,S75 - Management of ESBL infections in the pos...,https://online.eccmid.org/programme-live-1?cod...,,,Co-organised with \nMSD,15/03/2023,15:30,16:00,Industry Track,S75 - Management of ESBL infections in the pos...,S2,Session,Industry SessionIndustry Track,11. Other,,
3,,,S76 - SeluxDX,https://online.eccmid.org/programme-live-1?cod...,,,,16/03/2023,15:00,15:30,Industry Track,S76 - SeluxDX,S3,Session,Industry SessionIndustry Track,11. Other,,
4,,,S4 - MDx portfolio expansion: a new paradigm,https://online.eccmid.org/programme-live-1?cod...,Makeda SEMRET,Canada,Organized by \nSeegene Inc....,15/04/2023,12:15,13:15,Hall D,S4 - MDx portfolio expansion: a new paradigm,S4,Session,Integrated SymposiumHall D,11. Other,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,S1402,,How rapid AST solutions can be economically va...,https://online.eccmid.org/programme-live-1?cod...,,,,18/04/2023,,,Hall D,S14 - Rapid AST on positive blood culture: eff...,S55,Abstract,Integrated SymposiumHall D,11. Other,,
72,S1403,,Use of rapid AST solutions in different labora...,https://online.eccmid.org/programme-live-1?cod...,,,,18/04/2023,,,Hall D,S14 - Rapid AST on positive blood culture: eff...,S55,Abstract,Integrated SymposiumHall D,11. Other,,
73,S1404,,Laboratory Workflow Innovation for Rapid AST,https://online.eccmid.org/programme-live-1?cod...,,,,18/04/2023,,,Hall D,S14 - Rapid AST on positive blood culture: eff...,S55,Abstract,Integrated SymposiumHall D,11. Other,,
74,S1405,,Panel discussion and Q&A,https://online.eccmid.org/programme-live-1?cod...,,,,18/04/2023,,,Hall D,S14 - Rapid AST on positive blood culture: eff...,S55,Abstract,Integrated SymposiumHall D,11. Other,,


In [50]:
df.to_excel('industry_noFilter.xlsx', index=False)

### filter industry

In [43]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_number':[],
    'name':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

dates = ['02-16', '03-15', '03-16', '04-15', '04-16', '04-17', '04-18']

In [ ]:
import requests

cookies = {
    'PHPSESSID': 'lbwww6~2f22d77a3de3da12104acedee740dc52',
    'joe-chnlcustid': '0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
}

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    # 'Cookie': 'PHPSESSID=lbwww6~2f22d77a3de3da12104acedee740dc52; joe-chnlcustid=0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
    'Referer': 'https://online.eccmid.org/programme-live-1?coday=2023-04-18&embed=&typeHideAllBut=15,11',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'coday': '2023-04-18',
    'embed': '',
    'typeHideAllBut': '04-16',
    'advSearchThemeList': '70,71,72,73,74,75,76,89,90,91,92,93,94,77,78,79,80,81,82,83,84,85,86,87,88,95,96,97,98,99,47,48,49,50,51,52,53,54,55,24,25,26,27,28,29,30,31,2,3,4,5,6,7,8,9,32,33,34,35,36,37,38,39,40,41,56,57,58,59,60,61,42,43,44,45,46,62,63,64,65,66,67,68,69,10,11,12,13,14,15,16,17,18,19,20,21,22,23',
    '_': '1676228459225',
}

response = requests.get('https://online.eccmid.org/programme-live-1', params=params, cookies=cookies, headers=headers)

In [ ]:
soup = BS(response.content, 'lxml')

outer_row = soup.find_all(class_='row live-program-card-row-grid')
locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')

for ou in outer_row:

    col_div = 0

    for i_id in ou.find_all(class_='border text-center program-grid-calandar'):
        
        print(i_id)

In [ ]:
se_no = 1
for dat in dates:
    print(dat)
    reqout = requests.get(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&typeHideAllBut=15,11',
                         headers=headers,
                         cookies=cookies)

    for fil in range(2, 100):
        
        params = {
            'coday': '2023-04-18',
            'embed': '',
            'typeHideAllBut': '15,11',
            'advSearchThemeList': f'{fil}'
        }

        reqout = requests.get('https://online.eccmid.org/programme-live-1', params=params, headers=headers, cookies=cookies)
        soup = BS(reqout.content, 'lxml')

        outer_row = soup.find_all(class_='row live-program-card-row-grid')
        locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')

        for ou in outer_row:

            col_div = 0

            for i_id in ou.find_all(class_='border text-center program-grid-calandar'):

                if i_id.find(class_='programme-session-card programme-session-display border'):

                    session_number = f'S{se_no}'
                    print(f'=====**====={session_number}=====**=======', col_div)
                    i_id = i_id.find(class_='programme-session-card programme-session-display border')

                    data['sessionRef'] = i_id.get('data-session-reference')

                    req = requests.post(
                        'https://online.eccmid.org/fo-user-display-session-details.php',
                        cookies=cookies,
                        headers=headers,
                        data=data)

                    soupin = BS(req.content, 'lxml')
                    sess_data = soupin.find(id='modal-session-content')

                    # -------------------------------------    date time location ya type     ------------------------------------
                    sess_date = ''
                    sess_time = ''
                    sess_loc = ''
                    sess_type = ''
                    sess_loc = ''
                    sess_loc = locations[col_div].text.strip()
                    date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                    sess_date = date_time_type.split(',')[0].strip()
                    sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                    sess_type = date_time_type.split('CET')[-1].strip()

                    # --------------------------------------  category or type and title -----------------------------------------------------
                    sess_cate = ''
                    sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                    sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                    sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                    # --------------------------------------- abstract text OR co organised ---------------------------------------------
                    co_org = ''
                    sess_text = ''
                    if sess_data.find(class_='modal-session-organized-by mt-2'):
                        co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                    if sess_data.find(class_='modal-session-desc mt-2'):
                        sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()

                    sess_text = co_org + ' ' + sess_text

                    # ------------------------------------------- session ka authors aff --------------------------------------------------
                    sess_author = ''
                    sess_aff = ''

                    if sess_data.find(class_='modal-session-moderators'):
                        for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                            aff = i.find(class_='modal-session-moderator-country').text.strip()
                            auth = i.text.strip()
                            auth = auth.replace(aff, '').split('-')[0]
                            aff = aff.replace(',', '').strip()

                            sess_author += auth + '; '
                            sess_aff += aff + '; '

                    sess_author = clean_semicolon(sess_author)
                    sess_aff = clean_semicolon(sess_aff)
                    sess_date = sess_date + '/2023'
                    sess_type = re.sub(sess_loc, '', sess_type)
                    
                    print(f'sess date :- {sess_date}')
                    print(f'sess time :- {sess_time}')
                    print(f'sess loca :- {sess_loc}')
                    print(f'sess type :- {sess_type}')
                    print(f'sess cate :- {sess_cate}')
                    print(f'sess title :- {sess_title}')
                    print(f'abs text :- {sess_text}')
                    print(f'sess auth :- {sess_author}')
                    print(f'sess aff :- {sess_aff}')

                    dic['session_number'].append(session_number)
                    dic['name'].append('Session')
                    dic['source_id'].append('')
                    dic['manual_id'].append('')
                    dic['article_title'].append(sess_title)
                    dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                    dic['authors'].append(sess_author)
                    dic['author_affiliation'].append(sess_aff)
                    dic['abstract_text'].append(sess_text)
                    dic['date'].append(sess_date)
                    dic['start_time'].append(sess_time.split('-')[0].strip())
                    dic['end_time'].append(sess_time.split('-')[-1].strip())
                    dic['location'].append(sess_loc)
                    dic['session_title'].append(sess_title)
                    dic['session_type'].append(sess_type)
                    dic['category'].append(sess_cate)
                    dic['sub_category'].append('')
                    dic['disclosure'].append('')
                    print('======'*20)

                    # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*

                    abs_data = sess_data.find(class_='modal-sessions-interventions')
                    if abs_data:

                        pno = 1
                        for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                            abs_title = ab.find(style='font-weight: bold').text.strip()

                            # abs author
                            abs_author = ''
                            abs_aff = ''
                            if ab.find(class_='modal-session-faculties mb-1'):
                                ab_authaff = ab.find(class_='modal-session-faculties mb-1').text.strip()
                                abs_author = ', '.join(ab_authaff.split(',')[:-1])
                                abs_aff = ab_authaff.split(',')[-1]

                            if abs_title == '':
                                abs_title = 'TBD'

                            # source id...... time shit

                            ar_id = ''
                            art_time = ''

                            if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                                src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                                if src_code:

                                    if re.search('^\d{1,6}', src_code):
                                        ar_id = src_code + '-' + str(pno)
                                        art_time = ''
                                    elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                        str_pno = str(pno)
                                        if len(str_pno) == 1:
                                            str_pno = '0' + str_pno
                                        ar_id = src_code + str_pno
                                        art_time = sess_time
                            else:
                                ar_id = 'QC'


                            print(ar_id, '===', art_time)
                            print(f'abs title :- {abs_title}')
                            print(f'abs auth :- {abs_author}')
                            print(f'abs aff :- {abs_aff}')

                            dic['session_number'].append(session_number)
                            dic['name'].append('Presentation')
                            dic['source_id'].append(ar_id)
                            dic['manual_id'].append('')
                            dic['article_title'].append(abs_title)
                            dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                            dic['authors'].append(abs_author)
                            dic['author_affiliation'].append(abs_aff)
                            dic['abstract_text'].append('')
                            dic['date'].append(sess_date)
                            dic['start_time'].append(sess_time.split('-')[0].strip())
                            dic['end_time'].append(sess_time.split('-')[-1].strip())
                            dic['location'].append(sess_loc)
                            dic['session_title'].append(sess_title)
                            dic['session_type'].append(sess_type)
                            dic['category'].append(sess_cate)
                            dic['sub_category'].append('')
                            dic['disclosure'].append('')
                            print('-*-+-'* 17)
                            pno += 1

                    se_no += 1
                    print('inr', se_no)

                col_div += 1
            